<a href="https://colab.research.google.com/github/Moe-Khalaf/AdapMod_ML_ECED4676/blob/main/AdaptiveModSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptive Modulation Simulation
##### Developers: Mohammedali Khalaf, Rehan Khalid
## 1. Description
This Jupyter Notebook contains the project for ECED 4676. The objective of this
project is to model an adaptive modulation communication link.

This communication link is to be catered towards underwater communications, and as such, will be dealing with multipath and additive white gaussian noise. The particular schemes to be featured in this adaptive modulation link are
- B-FSK
- 4-FSK
- 8-FSK

B-FSK will be used to accomodate the harshest of conditions while 8-FSK will be used to accomodate the most favorable.

The way that this model will work is by assessing the bit error rate and time to transmit of a particular bitstream. Afterwards, this model will repeat the process using a fixed modulation system of each of the three selected modulation schemes. This way, the effectiveness of the adaptive modulation can be assessed against its fixed alternatives.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Function Library
Below are some user defined functions that will streamline the development process.

### 2.1 Bitstream Generation
The functions below are used in the process of generating bitstreams to test the system on.

In [27]:
def generate_random_bits(len):
    """
    Generate a bitstream to transmit.

    :param len: Length of the bitstream.
    :return: NumPy array containing the bitstream.
    """
    return np.array([random.randint(0, 1) for i in range(len)])

def convert_bitstream_for_4fsk(bitstream):
    """
    Convert a bitstream to a format suitable for 4FSK modulation. Essentially,
    this function groups the bits into pairs. [0, 1, 1, 1] -> [01, 11]

    :param bitstream: Bitstream to convert.
    :return: Converted bitstream.
    """
    # Convert the NumPy array to a string
    bitstream_str = ''.join(map(str, bitstream))

    # Group the bits into pairs
    paired_bits = [bitstream_str[i:i+2] for i in range(0, len(bitstream_str), 2)]

    return paired_bits

def convert_bitstream_for_8fsk(bitstream):
    """
    Convert a bitstream to a format suitable for 8FSK modulation. Essentially,
    this function groups the bits into triplets. [0, 1, 1, 1, 0, 1, 0, 1, 0] -> [011, 101, 010]

    :param bitstream: Bitstream to convert.
    :return: Converted bitstream
    """
    # Convert the NumPy array to a string
    bitstream_str = ''.join(map(str, bitstream))

    # Group the bits into triplets
    triplet_bits = [bitstream_str[i:i+3] for i in range(0, len(bitstream_str), 3)]

    return triplet_bits

def calculate_bitstream_errors(bitstream1, bitstream2):
    """
    Calculate the number of differing bits (errors) between two bitstreams.

    :param bitstream1: First bitstream as a numpy array.
    :param bitstream2: Second bitstream as a numpy array.
    :return: Number of differing bits (errors).
    """
    if bitstream1.shape != bitstream2.shape:
        raise ValueError("Bitstreams must be of the same length")

    # Calculate the number of differing bits
    errors = np.sum(bitstream1 != bitstream2)
    return errors

### 2.2. Signal Generation
The functions below are used in the process of generating signals.

In [31]:
def generate_BFSK_Signal_vectorized(bitstream, f1, f2, fs, fc, T_symbol):
    """
    Generate a BFSK signal using vectorization.

    :param bitstream: Bitstream to modulate.
    :param f1: Frequency representing '0'.
    :param f2: Frequency representing '1'.
    :param fs: Sampling frequency.
    :param T_symbol: Symbol duration.
    :return: BFSK signal as a numpy array.
    """
    num_bits = len(bitstream)
    t = np.arange(0, num_bits * T_symbol, 1/fs)  # Full time vector for all symbols

    # Initialize the waveform
    f_waveform = np.zeros(len(t), dtype=complex)

    # Create the waveform for each bit in the bitstream
    for i, bit in enumerate(bitstream):
        start_index = i * int(T_symbol * fs)
        end_index = start_index + int(T_symbol * fs)
        frequency = f1 if bit == 0 else f2
        f_waveform[start_index:end_index] = np.exp(2j * np.pi * frequency * t[start_index:end_index])

    # Upconvert with Carrier Frequency
    signal_upconverted = f_waveform*np.exp(2j*np.pi*fc*t)
    return signal_upconverted

def generate_4FSK_Signal(bitstream, f1, f2, f3, f4, fs, T_symbol):
    """
    Generate a 4FSK signal.

    :param bitstream: Bitstream to modulate.
    :param f1: Frequency representing '00'.
    :param f2: Frequency representing '01'.
    :param f3: Frequency representing '10'.
    :param f4: Frequency representing '11'.
    :param fs: Sampling frequency.
    :param T_symbol: Symbol duration.
    :return: 4FSK signal as a numPy array.
    """
    Converted_bitstream = convert_bitstream_for_4fsk(bitstream)
    t = np.arange(0, T_symbol, 1/fs) # Time Vector per Symbol (Start, Stop, Step)
    signal = np.array([])
    for bit in Converted_bitstream:
        if bit == '00':
            f = f1
        elif bit == '01':
            f = f2
        elif bit == '10':
            f = f3
        else:
            f = f4
        waveform = np.sin(2*np.pi*f*t)
        signal = np.concatenate((signal, waveform))
    return signal

def generate_8FSK_Signal(bitstream, f1, f2, f3, f4, f5, f6, f7, f8, fs, T_symbol):
    """
    Generate a 8FSK signal.

    :param bitstream: Bitstream to modulate.
    :param f1: Frequency representing '000'.
    :param f2: Frequency representing '001'.
    :param f3: Frequency representing '010'.
    :param f4: Frequency representing '011'.
    :param f5: Frequency representing '100'.
    :param f6: Frequency representing '101'.
    :param f7: Frequency representing '110'.
    :param f8: Frequency representing '111'.
    :param fs: Sampling frequency.
    :param T_symbol: Symbol duration.
    :return: 8FSK signal as a numPy array.
    """
    Converted_bitstream = convert_bitstream_for_8fsk(bitstream)
    t = np.arange(0, T_symbol, 1/fs) # Time Vector per Symbol (Start, Stop, Step)
    signal = np.array([])
    for bit in Converted_bitstream:
        if bit == '000':
            f = f1
        elif bit == '001':
            f = f2
        elif bit == '010':
            f = f3
        elif bit == '011':
            f = f4
        elif bit == '100':
            f = f5
        elif bit == '101':
            f = f6
        elif bit == '110':
            f = f7
        else:
            f = f8
        waveform = np.sin(2*np.pi*f*t)
        signal = np.concatenate((signal, waveform))
    return signal

### 2.3 Channel Condition Generation
The functions below are used in the process of generatng and applying channel conditions.

In [ ]:
def apply_multipath(signal, delays, attenuations, sampling_freq):
    """
    Apply multipath effects to a signal without extending its length.

    :param signal: The original signal (numpy array).
    :param delays: List of delays for each path in seconds.
    :param attenuations: List of attenuation factors for each path (0 to 1).
    :param sampling_freq: Sampling frequency of the signal.
    :return: Signal with multipath effects applied.
    """
    multipath_signal = np.copy(signal)  # Start with the original signal

    # Add each delayed and attenuated path
    for delay, attenuation in zip(delays, attenuations):
        delay_samples = int(delay * sampling_freq)
        delayed_signal = np.zeros(len(signal))
        delayed_signal[:len(signal) - delay_samples] = signal[delay_samples:] * attenuation
        multipath_signal += delayed_signal

    return multipath_signal

def apply_awgn_snr(signal, snr_db): # Applies addiive white gausian noise to signal
    """
    Apply Additive White Gaussian Noise to a signal based on a given SNR in dB.

    :param signal: The original signal (numpy array).
    :param snr_db: Desired Signal-to-Noise Ratio in dB.
    :return: Signal with AWGN applied.
    """
    # Calculate signal power
    signal_power = np.mean(np.abs(signal)**2)

    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_db / 10)

    # Calculate noise power based on SNR
    noise_power = signal_power / snr_linear

    # Generate white Gaussian noise
    noise = np.random.normal(0, np.sqrt(noise_power), len(signal))

    # Add noise to the signal
    noisy_signal = signal + noise

    return noisy_signal

def generate_random_mp_conditions():
    """
    Generate a random set of multipath conditions with ordered delays and attenuations scaling linearly with delay.

    :return: List of multipath conditions.
    """
    alpha = -0.02645 # Attenuation constant (Approximation based of of kinslers fundamentals of acoustics)
    # Generate a random number of paths
    num_paths = np.random.randint(1, 5)

    # Generate and sort random delays
    delays = np.sort(np.random.uniform(0, 0.01, num_paths))

    # Calculate attenuations that decay exponentially with delay
    attenuations = [min_attenuation * np.exp(-delay * alpha/max(delays)) for delay in delays]

    # Ensure that attenuations do not exceed min_attenuation
    attenuations = [max(att, min_attenuation) for att in attenuations]

    return delays, attenuations

### 2.4 Demodulation
The functions below are used in the process of demodulating signals.

In [32]:
def matched_filter(signal, template, fs):
    """
    Apply a matched filter to the signal.

    :param signal: The input signal.
    :param template: The template or reference signal for the matched filter.
    :param fs: Sampling frequency.
    :return: The filtered signal.
    """
    return np.correlate(signal, template, 'same')

def demodulate_BFSK_FrequencyMixing_Vectorized(bfsk_signal, f1, f2, fs, fc, T_symbol):
    num_samples_per_symbol = int(fs * T_symbol)
    num_symbols = len(bfsk_signal) // num_samples_per_symbol

    # Create a time vector for the entire signal
    t = np.arange(0, len(bfsk_signal) / fs, 1 / fs)

    # Downconvert the signal to baseband
    downconverted_signal = bfsk_signal * np.exp(-2j * np.pi * fc * t)

    # Reshape the downconverted signal to a 2D array (each row is a symbol)
    reshaped_signal = downconverted_signal[:num_symbols * num_samples_per_symbol].reshape((num_symbols, num_samples_per_symbol))

    # Create a time vector for one symbol and repeat it for each symbol
    t_symbol = np.arange(0, T_symbol, 1 / fs)
    t_repeated = np.tile(t_symbol, (num_symbols, 1))

    # Vectorized frequency mixing at BFSK frequencies
    fhe = np.exp(2j * np.pi * f2 * t_repeated) * reshaped_signal
    fle = np.exp(2j * np.pi * f1 * t_repeated) * reshaped_signal

    # Vectorized integration and decision making
    power_0 = np.abs(np.sum(fhe, axis=1)**2)
    power_1 = np.abs(np.sum(fle, axis=1)**2)
    bitstream = (power_1 > power_0).astype(int)

    return bitstream

## 3. System Parameters
Below are some of the parameters that will be used within the communication link. The parameters will include but are not limited to some characteristics such as the communication band, sampling frequency, symbol period, an array of multipath conditions, and an array of signal to noise ratios.

In [ ]:
Signal_Bandmin = 25000
Signal_Bandmax = 30000
Sampling_Frequency = 100000
Symbol_Period = 0.02 # 20ms

Test

In [34]:
def test_modulation_demodulation(n, bitstream_length):
    error_counts = []
    for _ in range(n):
        test_bitstream = generate_random_bits(bitstream_length)
        test_signal = generate_BFSK_Signal_vectorized(test_bitstream, -2500, 2500, 100000, 27500, 0.02)
        test_result = demodulate_BFSK_FrequencyMixing_Vectorized(test_signal, -2500, 2500, 100000, 27500, 0.02)
        err = calculate_bitstream_errors(test_bitstream, test_result)
        error_counts.append(err)
    return error_counts

# Example usage
n_tests = 100  # Number of tests
bitstream_length = 100  # Length of each bitstream
errors = test_modulation_demodulation(n_tests, bitstream_length)
print(f"Errors in each test: {errors}")

Errors in each test: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
